In [1]:
import numpy
import json
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1", do_lower_case=True)
model = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1")


def get_embeddings(text):
        tokenized_text = tokenizer(text, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**tokenized_text)

        word_embeddings = outputs.last_hidden_state
        averaged_embedding = torch.mean(word_embeddings, dim=0)

        return averaged_embedding


/Users/inttstbrd/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

json_dict = {}

def json_creator_context(cursor):
    cursor.execute("select treeId, internalRep from Meaning")
    i = 0
    for row in cursor:
        i+=1
        # print(i, " / 384863")
        if i > 4:
            break
        
        # for storing embedings use numpy.array(get_embeddings(row[1])) with , cls=NumpyArrayEncoder at json.dump
        if row[0] not in json_dict.keys():
            json_dict[row[0]] = [row[1]]
        else:
            json_dict[row[0]].append(row[1])
    


In [6]:
import feather
import pyarrow as pa

def stocare_eficienta(data_dict, filename="data.feather"):
    """
    Stochează datele în format Feather.
    """
    try:
        comprimat_dict = {}
        for cheie, matrice_list in data_dict.items():
            comprimat_dict[cheie] = [comprima_matrice(matrice) for matrice in matrice_list]
        
        print(comprimat_dict)
        # schema = pa.Schema([
        #     pa.field()
        # ])
        # table = pa.Table.from_pydict(data_dict, schema=schema)

        # feather.write_dataframe(table, filename)
        print(f"Date salvate cu succes în {filename}")
    except Exception as e:
        print(f"Eroare la stocarea datelor: {e}")

def comprima_matrice(matrice):
    """
    Compresează o matrice NumPy folosind zstandard.
    """
    import zstandard as zstd
    return zstd.compress(matrice.tobytes())

def incarcare_eficienta(filename="data.feather"):
    """
    Încarcă datele din format Feather.
    """
    try:
        comprimat_dict = feather.read_dataframe(filename)
        decomprimat_dict = {}
        for cheie, matrice_comprimate in comprimat_dict.items():
            decomprimat_dict[cheie] = [decomprima_matrice(matrice) for matrice in matrice_comprimate]
        return decomprimat_dict
    except Exception as e:
        print(f"Eroare la încărcarea datelor: {e}")
        return None

def decomprima_matrice(comprimat):
    """
    Decomprimă o matrice NumPy comprimată cu zstandard.
    """
    import zstandard as zstd
    return numpy.frombuffer(zstd.decompress(comprimat), dtype=numpy.float32)

stocare_eficienta(json_dict)


{}
Date salvate cu succes în data.feather
